In [3]:
import tensorflow as tf
import numpy as np

In [4]:
with np.load("notMNIST.npz") as data :
    Data, Target = data ["images"], data["labels"]
    posClass = 2
    negClass = 9
    dataIndx = (Target==posClass) + (Target==negClass)
    Data = np.reshape(Data, [-1, 28*28])
    Data = Data[dataIndx]/255.
    Target = Target[dataIndx].reshape(-1, 1)
    Target[Target==posClass] = 1
    Target[Target==negClass] = 0
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data, Target = Data[randIndx], Target[randIndx]
    trainData, trainTarget = Data[:3500], Target[:3500]
    validData, validTarget = Data[3500:3600], Target[3500:3600]
    testData, testTarget = Data[3600:], Target[3600:]

In [21]:
train_size, num_feature = np.shape(trainData)
num_label = 1

In [22]:
x = tf.placeholder("float", shape=[None, num_feature])
y_true = tf.placeholder("float", shape=[None, num_label])

W = tf.Variable(tf.zeros([num_feature, num_label]))
b = tf.Variable(tf.zeros([num_label]))
y = tf.matmul(x, W) + b

In [23]:
weight_decay = 0.0

In [52]:
#loss = 0.5*tf.reduce_mean(tf.squared_difference(y, y_true)) + 0.5*weight_decay*tf.reduce_sum(tf.square(W))
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y)) + 0.5*weight_decay*tf.reduce_sum(tf.square(W))

In [53]:
learning_rate = 0.005
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [54]:
delta = tf.abs((y_true - y))
correct_prediction = tf.cast(tf.less(tf.constant(0.5), delta), tf.int32)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [55]:
batch_size = 500
iteration = 200

with tf.Session() as s:
    tf.global_variables_initializer().run()
    
    for step in xrange(iteration):
        offset = (step*batch_size) % num_label
        
        batch_data = trainData[offset:(offset + batch_size), :]
        batch_target = trainTarget[offset:(offset + batch_size)]
    
        train_step.run(feed_dict={x: batch_data, y_true: batch_target})
    

    print accuracy.eval({x: testData, y_true: testTarget})
    print s.run(loss, feed_dict={x: trainData, y_true: trainTarget})
    print s.run(tf.norm(W))
    print s.run(tf.reduce_sum(tf.square(W)))

0.94482756
0.12000771
0.5818035
0.33849528


In [42]:
with np.load("notMNIST.npz") as data:
    Data, Target = data ["images"], data["labels"]
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data = Data[randIndx]/255.
    Target = Target[randIndx]
    trainData, trainTarget = Data[:15000], Target[:15000]
    validData, validTarget = Data[15000:16000], Target[15000:16000]
    testData, testTarget = Data[16000:], Target[16000:]

In [44]:
trainTarget

array([5, 9, 9, ..., 2, 0, 9], dtype=uint8)